In [1]:
#Save the BLAST output file for each sample, open jupyter notebook for processing
#Import pandas to process the reads into QIIME2 compatible format
import pandas as pd
import os

In [2]:
#set the base directory to the blast output folder
basedir = '/mnt/c/MinION_16S_barcodes/BLASTn/BLASTn/full_BLAST/'

In [3]:
taxa = os.path.join(basedir, 'hare_136_sample13_21_barcode01.fa.SILVA.besthit.outfmt6')

In [4]:
#read the BLAST output file assigning the coloumn headers
taxa_df = pd.read_csv(taxa, sep= '\t', names = ["Query Seq-id","Subject Seq-id","evalue","bitscore","length","pident","nident","subject-GI","Subject_accession","Subject_Taxonomy_ID","Start_of_alignment","End_of_alignment"])

In [6]:
#Drop the columns that are not required in the dataframe
taxa_df.drop(columns=['Subject Seq-id', 'evalue', 'bitscore', 'length', 'pident','nident','subject-GI','Subject_Taxonomy_ID','Start_of_alignment','End_of_alignment'], inplace=True)

In [8]:
#The subject_accession coloumn contains multiple accession numbers separated by ".". Split the column into three using "." and assign it to a new dataframe
taxa_df_1 = taxa_df["Subject_accession"].str.split(".",n=2,expand = True)

In [10]:
#rename the column
taxa_df_1.columns = ['A', 'B', 'C']

In [12]:
#concatenate the the old and the new dataframe together
result = pd.concat([taxa_df, taxa_df_1], axis=1)

In [14]:
#Only the first part of the subject_accession contains the main accession number. Retain only that and remove other columns
result.drop(columns=['Subject_accession', 'B','C'], inplace=True)

In [16]:
#Rename the column
result.rename(columns={'A': 'primaryAccession'}, inplace=True)

In [18]:
#Number of rows with unique accession number
result['primaryAccession'].nunique()

19774

In [110]:
#Use pivot table to group all the reads (Query Seq-id) with the same accession number. 
result_df=pd.pivot_table(result, values = 'Query Seq-id',index=['primaryAccession'],aggfunc=len,dropna=False)

In [111]:
#Rename the column to sample ID
result_df.rename(columns={'Query Seq-id': 'MF-136'}, inplace=True)

In [120]:
#Save the file as tsv
result_df.to_csv(os.path.join(basedir, '20190724_mf136_pivot_with_accessionnumber.tsv'),sep='\t',header=True,index =True)